# Getting started

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import ompy as om
import logging

In [3]:
om.__full_version__

'0.4.0.dev0+da5bf4d'

In [4]:
# get smaller files for the online version
plt.rcParams["figure.dpi"] = 70

## Loading and example raw spectra

In [5]:
# Import raw matrix into instance of om.Matrix() and plot it
raw = om.example_raw('Dy164')
# To use you own data, uncomment/adapt the line below instead
# raw = om.Matrix(path="/path/to/matrix.ending")

# Plot the entire matrix
raw.plot();

# Note: We use the semi-colon `;` at the end of the line to silence the output
# in jupyter notebook. This is not necessary, but otherwise you get something like 
# this below printed every time:
#(<matplotlib.collections.QuadMesh at 0x7fafbc422eb8>,
# <matplotlib.axes._subplots.AxesSubplot at 0x7fafc0944a20>,
# <Figure size 640x480 with 2 Axes>)

<IPython.core.display.Javascript object>

## Matrix manipulation

The core of the Oslo method involves working with two dimensional spectra. Starting with a raw matrix of $E_x$-$E_\gamma$ coincidences, you typically want to unfold the counts
along the gamma-energy axis and then apply the first-generation method to obtain the matrix of first-generation, or primary, gamma rays from the decaying nucleus.

The two most important utility classes in the package are `Matrix()` and `Vector()`. They are used to store matrices (2D) or vectors (1D) of numbers, typically spectra of counts, along with energy calibration information. 

As these underpin the entire package, they contain many useful functions to make life easier. Loading and saving to several formats, plotting, projections, rebinning and cutting, to mention a few. See the documentation for an exhaustive list.

Their basic structure is:

In [6]:
# mat = ompy.Matrix()
mat = raw
mat.values  # A 2D numpy array
mat.Ex      # Array of mid-bin energy values for axis 0 (i.e. the row axis, or y axis)
mat.Eg      # Array of mid-bin energy values for axis 1 (i.e. the column axis, or x axis)

print("The first gamma-ray energies:\n", mat.Eg[0:10])

The first gamma-ray energies:
 [  0.     19.364  38.728  58.092  77.456  96.82  116.184 135.548 154.912
 174.276]


In [7]:
# We can also create a vector, which is useful to store the NLD and gSF.
values = np.arange(11)
E = np.linspace(0, 10, num=11)

fig, ax = plt.subplots(figsize=(2,2), constrained_layout=True)
vec = om.Vector(values=values, E=E)
vec.values  # A 1D numpy array
vec.E       # Array of lower-bin-edge energy values for the single axis
vec.plot(ax=ax);

<IPython.core.display.Javascript object>

In [8]:
# Cut away counts above the diagonal 
# Remember: Think about what you do here. If you cut them away, they will not
# be used in unfolding etc. This may or may not be what you want.
# Note that the raw matrix we read in above has been cut already, so the difference here is not so large.
raw.cut_diagonal(E1=(800, 0), E2=(7500, 7300))
raw.cut('Ex', 0, 8400)
raw.plot();

<IPython.core.display.Javascript object>

Note that `Matrix`, `Vector` and several other classes contain mutable objects. If you work on them, you might want to create a *deepcopy*. For `Matrix`, `Vector` this can be archived by the convince method `X.copy`, otherwise use `copy.deepcopy`.

In [9]:
# The "right" way if you don't want to change the original matrix
raw_big_cut = raw.copy()
raw_big_cut.cut('Ex', 0, 4000)
print(raw.Ex.max(), raw_big_cut.Ex.max())

8300.0 3980.0


In [10]:
# The "wrong" way if you don't want to change the original matrix
raw_big_cut2 = raw_big_cut
raw_big_cut2.cut('Ex', 0, 2000)
print(raw_big_cut.Ex.max(), raw_big_cut2.Ex.max())
# oups!: suddenly also `raw_big_cut` was cut, not only raw_big_cut2

1940.0 1940.0


In [11]:
# Plot projections
raw.plot_projection('Ex', Emin=1800, Emax=2600);

<IPython.core.display.Javascript object>

Note that you can IPython's has tools to quickly access information on a function, namely the `?` character to explore documentation, the `??` characters to explore source code, and the `Tab key` (or `double-tab`) for auto-completion. Try it out uncommenting the function below.

In [12]:
## Uncomment these lines to query a function
# ?raw.plot_projection

## Unfolding

### Get a response matrix

In [13]:
logger = om.introspection.get_logger('response', 'INFO')
# Then do the same using OMpy functionality:
# You may need to adpot this to whereever you response matrixes are stored
folderpath = "../oscar_response/oscar2017_scale1.15"

# Energy calibration of resulting response matrix:
Eg = raw.Eg

# Experimental relative FWHM at 1.33 MeV of resulting array
fwhm_abs = 30 # (30/1330 = 2.25% )

# Magne recommends 1/10 of the actual resolution for unfolding purposes
R_ompy_view, R_tab_view = om.interpolate_response(folderpath, Eg, fwhm_abs=fwhm_abs, return_table=True)
R_ompy_unf, R_tab_unf = om.interpolate_response(folderpath, Eg, fwhm_abs=fwhm_abs/10, return_table=True)

2019-10-11 17:01:04,889 - ompy.response - INFO - Note: The response below 200 keVis interpolation only, as there are no simulations available.
2019-10-11 17:01:10,513 - ompy.response - INFO - Note: The response below 200 keVis interpolation only, as there are no simulations available.


In [14]:
# You can decide to log information and set the logging level (info/debug)
logger = om.introspection.get_logger('unfolder', 'INFO')

# We need to remove negative counts (unphysical) in the raw matrix before unfolding:
raw_positive = raw.copy()
raw_positive.fill_and_remove_negative(window_size=2)

# With compton subtraction and all tweaks
unfolder= om.Unfolder(response=R_ompy_unf)
unfolder.use_compton_subtraction = True # default
unfolder.response_tab = R_tab_unf
# Magne suggests some "tweaks" for a better unfolding performance. Default is 1 for all.
unfolder.FWHM_tweak_multiplier = {"fe": 1., "se": 1.1,
                                     "de": 1.3, "511": 0.9}
unfolded = unfolder(raw_positive)
unfolded.plot();

<IPython.core.display.Javascript object>

In [15]:
### Generate the first generation matrix

In [16]:
firstgen = om.FirstGeneration()
primary = firstgen(unfolded)
primary.plot();

<IPython.core.display.Javascript object>

## Propagating statistical uncertainties

In order to propagate the statistical uncertainties from the raw matrix, we use an ensemble based method. We start of my generating en enseble of *raw-like* matrixes. The raw counts are poisson distributed (actually, they are so before background subtraction, see issue on github). If we had counted one another time, we would get slightly different results. 

We take the number of counts $k_i$ in bin $i$ of the raw matrix $R$ as an estimate for the Poisson parameter ("the mean") $λ_i$ . Note that it is an unbiased estimator for $λ_i$, since $E(k) = λ$. To generate a member matrix $R_l$ of the MC ensemble, we replace the counts in each bin $i$ by a random draw from the distribution $\operatorname{Poisson}(k_i)$.

The class Ensemble() provides this feature. Its basic usage is:

In [29]:
raw.values[raw.values<0]

array([-1.06666737e+01, -3.99999809e+00, -5.49999714e+00, -1.33333135e+00,
       -2.83333874e+00, -1.66666985e-01, -6.00001144e+00, -5.50001049e+00,
       -3.83334064e+00, -1.63333282e+01, -3.00000572e+00, -6.00001144e+00,
       -2.36666603e+01, -1.33333588e+00, -7.83334827e+00, -2.40000076e+01,
       -1.51666641e+01, -1.59999981e+01, -1.66665077e-01, -2.59999809e+01,
       -1.01666222e+01, -2.18333321e+01, -3.13333073e+01, -2.08333054e+01,
       -3.16665745e+00, -3.56666451e+01, -9.99975204e-01, -1.34999971e+01,
       -1.01666365e+01, -2.00000381e+00, -2.83333263e+01, -8.99999619e+00,
       -9.33333015e+00, -6.49998951e+00, -5.16664791e+00, -1.21666670e+01,
       -9.16666508e+00, -1.96666470e+01, -5.83332348e+00, -5.83333492e+00,
       -1.76666603e+01, -1.66666985e-01, -2.66666412e+00, -3.33314896e-01,
       -9.66663933e+00, -6.66666031e-01, -4.83331966e+00, -1.13333263e+01,
       -2.33332634e+00, -1.21666555e+01, -8.33333111e+00, -1.66666031e-01,
       -1.16667747e+00, -

In [39]:
%load_ext snakeviz

logger = om.introspection.get_logger('ensemble', 'INFO')

# Tell the `Ensemble` class which raw spectrum, what kind of undolfer and first
# generations method to use.
# Note: This will have the same setting as above. We could for example have
# set the first generations method to use a different "valley_collection", or a
# differnt type of "multiplicity_estimation"
ensemble = om.Ensemble(raw=raw_positive)
ensemble.unfolder = unfolder
ensemble.first_generation_method = firstgen
%snakeviz ensemble.generate(10, regenerate=True)    # Generates N perturbated members



  0%|          | 0/10 [00:00<?, ?it/s]

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
2019-10-11 17:09:41,638 - ompy.ensemble - INFO - Generating 0




 10%|█         | 1/10 [00:01<00:12,  1.34s/it]

2019-10-11 17:09:42,977 - ompy.ensemble - INFO - Generating 1




 20%|██        | 2/10 [00:02<00:10,  1.36s/it]

...

2019-10-11 17:09:52,480 - ompy.ensemble - INFO - Generating 8




 90%|█████████ | 9/10 [00:12<00:01,  1.35s/it]

2019-10-11 17:09:53,849 - ompy.ensemble - INFO - Generating 9




100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


 
*** Profile stats marshalled to file '/tmp/tmp07hht7qj'. 
Embedding SnakeViz in this document...


The generated members are saved to disk and can be retrieved. Unfolded members can be retrieved as `ensemble.get_unfolded(i)`, for example. Their standard deviation is `ensemble.std_unfolded` for the unfolded matrixes, etc.

We can now plot the standard deviation of all ensemble members for the raw, unfolded and first generation spectrum 

In [18]:
i_unfolded = 10
matrix = ensemble.get_unfolded(i_unfolded)
matrix.plot(title=f"Unfolded matrix #{i_unfolded}")

# Following commands plots all std. deviations
ensemble.plot();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Nuclear level density and gamma strength function

After matrix has been cut, unfolded and firstgen'd, perhaps ensembled, its nuclear level density (nld) and gamma strength function ($\gamma$SF) can be extracted using the `Extractor()` class.  

The method relies on the relation
 \begin{align}
	P(E_x, E_\gamma) \propto NLD(E_x - E_\gamma) \mathcal{T}(E_\gamma),\label{eq:Oslo_method_eq}
\end{align}
where $P(E_x, E_\gamma)$ is the first-generation spectrum normalized to unity for each $E_x$ bin.  
Furthermore, if we assume that the $\gamma$ decay at high $E_x$ is dominated by dipole radiation the transmission coefficient \mathcal{T} is related to the dipole $\gamma$-ray strength function $f(E_\gamma)$ by the relation
\begin{align}
    \mathcal{T}(E_\gamma) = 2\pi E_\gamma^3 f(E_\gamma).\label{eq:gammaSF}
\end{align} 

If you have reasons to assume a different multipose decomposition, you may of course calculate the transmission coefficient \mathcal{T} from the $\gamma$-ray strength function produced here and apply the decomposition you prefer.

For a single matrix, its usage is:  
(well, think about what you want to set in as the std. deviation)

In [19]:
# cutout = primary.trapezoid(Ex_min=4000, Ex_max=8000, Eg_min=1000, inplace=False)
# cutout_std = ensemble.std_firstgen.trapezoid(Ex_min=4000, Ex_max=8000, Eg_min=1000, inplace=False)
# extractor = om.Extractor()
# nld, gsf = extractor.decompose(cutout, std=cutout_std)

When extracting NLD and GSF from an ensemble, a trapezoidal cutout must be performed on each ensemble member. This is achieved by `Action()` which allows for delayed function calls on matrices and vectors. This way we don't cut the raw matrix at `Ex_min`, but this will only happen before the extraction.

In [20]:
trapezoid_cut = om.Action('matrix')
trapezoid_cut.trapezoid(Ex_min=4000, Ex_max=7000, Eg_min=1000, inplace=True)
extractor = om.Extractor()
extractor.trapezoid = trapezoid_cut
# Running the lines below directy, would most probably 
# result in a error like
# The AssertionError: Ex and Eg must have the same step size
#
# Why? The extraction assumes that Ex and Eg have the same binning. Thus we
# need to rebin the ensemble. This works will work inplace. 
# Note: As always, be careful will mid-bin vs lower bin calibration.
E_rebinned = ensemble.get_firstgen(0).Ex
ensemble.rebin(E_rebinned, member="firstgen")
ensemble.plot();

100%|██████████| 50/50 [00:00<00:00, 780.96it/s]


<IPython.core.display.Javascript object>

In [21]:
# now we can extract the NLD and gSF for N of the samples of the ensemble 
# (here just 10 to speed things up)
extractor.size = 10
extractor.extract_from(ensemble, regenerate=True)

 10%|█         | 1/10 [00:00<00:05,  1.55it/s]

Optimization terminated successfully.
         Current function value: 2727.988737
         Iterations: 4
         Function evaluations: 4430


 20%|██        | 2/10 [00:01<00:04,  1.63it/s]

Optimization terminated successfully.
         Current function value: 2713.106829
         Iterations: 4
         Function evaluations: 4402


 30%|███       | 3/10 [00:01<00:04,  1.67it/s]

...

Optimization terminated successfully.
         Current function value: 2639.712549
         Iterations: 4
         Function evaluations: 4413


 90%|█████████ | 9/10 [00:05<00:00,  1.61it/s]

Optimization terminated successfully.
         Current function value: 2587.251676
         Iterations: 4
         Function evaluations: 4411


100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

Optimization terminated successfully.
         Current function value: 2609.012167
         Iterations: 4
         Function evaluations: 4411


The resulting `nld` and `gsf` are saved to disk and exposed as `extractor.nld` and `extractor.gsf`

### Plotting the results before normalization

In [22]:
extractor.plot(plot_mean=False)

<IPython.core.display.Javascript object>

(<Figure size 448x336 with 2 Axes>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fb2b80dd6a0>,
       dtype=object))

Or maybe you are more used to displaying the results with std. deviations?

**Note**: This may be erroneous, as the nld and gsf are not normalized yet!  
Thus, in principal, we might evaluate std. devs. of the *same solution* with different  
transformations. Before we normalize, we don't know. And they have the same $\chi^2$.  
That was the reason for the *trouble* with normalization.



In [23]:
extractor.plot(plot_mean=True)

<IPython.core.display.Javascript object>

(<Figure size 448x336 with 2 Axes>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fb2b7f4da58>,
       dtype=object))

## Normalization

Does it still look *strange*? probably because you are only used to see the normalized results.

### 1) Manual normalization

In [24]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_transformed(alpha):
    fig, ax = plt.subplots(1, 2, constrained_layout=True)
    for nld, gsf in zip(extractor.nld, extractor.gsf):
        nld.transform(const=1, alpha=alpha, inplace=False).plot(ax=ax[0], scale="log", color='k', alpha=1/10)
        gsf.transform(const=1, alpha=alpha, inplace=False).plot(ax=ax[1], scale="log", color='k', alpha=1/10)
    ax[0].set_title("Level density")
    ax[1].set_title("γSF")

plot_transformed(alpha=0.0015)

<IPython.core.display.Javascript object>

### 2) Normalization (of nld) through external data 
Normalization of the $\gamma$SF will follow *very* soon. We just need to clean up the code.
For now we will provide a normalization routine for the NLD only; afterward this will be combined to a *global* normalization, i.e. $\gamma$SF and NLD will be normalized at the same time.

The normalization ensures that we find the *physical* solution, so we remove the degeneracy that is in principal inherent to decomposition of NLD and $\gamma$SF:
\begin{align}
NLD' = NLD(E_x) * A exp(\alpha E_x) \\
\gamma SF' = \gamma SF(E_\gamma) * B exp(\alpha E_\gamma)
\end{align}
Note: This is the transformation eq (3), Schiller2000.

As external data for the normalization we commonly use:
1. the discrete leves, binned with the resolution of our data (and potentially also smoothed)
2. The NLD at Sn, derived from D0 and a spin distribution
3. The average total radiative width $\Gamma_\gamma$.

#### Let's first normalize the mean from the extractor:
This part is **almost ready** -- but not well-tested yet. After some recent changes, the calculated D0, for example, seems to be strange

In [25]:
normlog = om.introspection.get_logger('normalizer', 'DEBUG')
nld_mean = om.Vector(values=extractor.nld_mean(), std=extractor.nld_std(), E=extractor.nld[0].E)
normalizer = om.Normalizer(nld=nld_mean, discrete='../example_data/discrete_levels_Dy164.txt')

# if you decide not to smooth the 
normalizer.use_smoothed_levels = False

2019-10-11 17:01:24,645 - ompy.normalizer - DEBUG - Set `discrete` by loading smooth
2019-10-11 17:01:24,645 - ompy.normalizer - DEBUG - Created Normalizer
2019-10-11 17:01:24,647 - ompy.normalizer - DEBUG - Set `discrete` by loading discrete


In [26]:
Sn = 7.658 # MeV
normalizer.spin['spincutModel'] = 'Disc_and_EB05' # see eg. Guttormsen et al., 2017, PRC 96, 024313
normalizer.spin['spincutPars'] = {"mass":164, "NLDa":18.12, "Eshift":0.31,
                                  "Sn": Sn, "sigma2_disc":[1.5,3.6]}
normalizer.spin['J_target'] = 0 # A-1 nucleus
normalizer.spin['Gg'] = [112, 20] # units
normalizer.spin['Sn'] = Sn
normalizer.D0 = [6.8, 0.6]
normalizer.normalize(limit_low=[0, 1.5], limit_high=[3, 5.5])

2019-10-11 17:01:24,787 - ompy.normalizer - INFO - 

---------
Normalizing nld #0
2019-10-11 17:01:24,788 - ompy.normalizer - DEBUG - Setting NLD, convert to MeV
2019-10-11 17:01:24,788 - ompy.normalizer - DEBUG - Using bounds [(1, 100), (0.1, 20), (0.1, 1), (6.732, 6.867999999999999)]
2019-10-11 17:01:24,789 - ompy.normalizer - DEBUG - Using spin {
    "Gg": [
        112,
        20
    ],
    "J_target": 0,
    "Sn": 7.658,
    "spincutModel": "Disc_and_EB05",
    "spincutPars": {
        "Eshift": 0.31,
        "NLDa": 18.12,
        "Sn": 7.658,
        "mass": 164,
        "sigma2_disc": [
            1.5,
            3.6
        ]
    }
}
2019-10-11 17:01:25,504 - ompy.normalizer - INFO - DE results:
┌───────────────────┬───────────────────┬────────────────────┬───────────────────┐
│ A                 │ α [MeV⁻¹]         │ T [MeV]            │ D₀ [eV]           │
╞═══════════════════╪═══════════════════╪════════════════════╪═══════════════════╡
│ 4.525692930431674 │ 1.8710968607

In [27]:
normalizer.plot();

<IPython.core.display.Javascript object>

#### Alternatively, we can normalize each member of the extractor ensemble separatly:

In [28]:
normalizer.normalize(extractor=extractor, limit_low=[0, 1.5], limit_high=[3, 5.5])

2019-10-11 17:01:46,527 - ompy.normalizer - INFO - 

---------
Normalizing nld #0
2019-10-11 17:01:46,527 - ompy.normalizer - DEBUG - Setting NLD, convert to MeV
2019-10-11 17:01:46,528 - ompy.normalizer - DEBUG - Using bounds [(1, 100), (0.1, 20), (0.1, 1), (6.732, 6.867999999999999)]
2019-10-11 17:01:46,528 - ompy.normalizer - DEBUG - Using spin {
    "Gg": [
        112,
        20
    ],
    "J_target": 0,
    "Sn": 7.658,
    "spincutModel": "Disc_and_EB05",
    "spincutPars": {
        "Eshift": 0.31,
        "NLDa": 18.12,
        "Sn": 7.658,
        "mass": 164,
        "sigma2_disc": [
            1.5,
            3.6
        ]
    }
}


KeyboardInterrupt: 

In [ ]:
normalizer.plot();